# Analyse the results using arviz

JAXspec provides a convenient way to analyse the results of a fit using the [`arviz`](https://python.arviz.org/en/stable/) library. This library provides powerful tool to explore Bayesian models. In this example, we will show how to use [`arviz`](https://python.arviz.org/en/stable/) to analyse the results of a fit.

In [1]:
import numpyro

numpyro.enable_x64()
numpyro.set_platform("cpu")
numpyro.set_host_device_count(4)

import numpyro.distributions as dist
from jaxspec.fit import BayesianFitter
from jaxspec.data import ObsConfiguration
from jaxspec.model.background import BackgroundWithError
from jaxspec.model.additive import Diskbb, Powerlaw
from jaxspec.model.multiplicative import Tbabs

model = Tbabs() * ( Diskbb() + Powerlaw() )

obs = ObsConfiguration.from_pha_file(
    'data/PN_spectrum_grp20.fits', 
    low_energy=0.3, 
    high_energy=7.5
)

forward = BayesianFitter(model, obs, background_model=BackgroundWithError())

prior = {
    "diskbb_1": {
        "Tin": dist.Uniform(0, 5), 
        "norm": dist.Exponential(1e4)
        }, 
    "powerlaw_1":{
        "alpha": dist.Uniform(0, 5), 
        "norm": dist.Exponential(1e4)
        }, 
    "tbabs_1": {
        "N_H": dist.Exponential(1e4)
        }
}

result = forward.fit(
    prior, 
    num_chains=4, 
    num_warmup=1000, 
    num_samples=1000,
    mcmc_kwargs={"progress_bar": True}
)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/Users/sdupourque/PycharmProjects/jaxspec/src/jaxspec/tables/apec.nc', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

From the result object, you can access the `inference_data` attribute, which is an `arviz.InferenceData` object. This leverage the use of every arviz function to analyse the results of the fit.

In [ ]:
inference_data = result.inference_data
inference_data

## Trace plot
This visualization is useful to see the evolution of the parameters during the sampling process. It can be used to diagnose convergence issues. The ideal situation is when the chains are well mixed and randomly scattered around the target distribution. If instead, chains are stuck in some region of the parameter space, or show some trends, this might indicate that the sampler did not explore the full parameter space.

In [ ]:
import arviz as az
import matplotlib.pyplot as plt 

with az.style.context("arviz-darkgrid", after_reset=True):
    az.plot_trace(inference_data, compact=False)

plt.show()

A more quantitative way to assess the convergence of the chains is to use the `summary` function. This function provides a summary of the posterior distribution of the parameters, including the mean, the standard deviation, and the 95% highest posterior density interval.

In [ ]:
az.summary(result.inference_data.posterior)

The `r_hat` column provides a measure of the convergence of the chains. The closer to 1, the better. A value larger than 1.1 is a sign of convergence issues. This statistic can be directly computed using the `r_hat` function, [see Vehtari et al. (2019)](https://arxiv.org/abs/1903.08008).

In [ ]:
rhat = az.rhat(result.inference_data.posterior)
rhat

## Pair plot

This visualization is useful to see the correlation between the parameters. The ideal situation is when the parameters are uncorrelated, which means that the posterior distribution is close to a multivariate Gaussian distribution.

In [ ]:
with az.style.context("arviz-darkgrid", after_reset=True):
    az.plot_pair(result.inference_data)
    
plt.show()

Take a look at [arviz's documentation](https://python.arviz.org/en/stable/examples/index.html) to see what else you can do with this library.